# モデルの最大文脈長を超える長さのテキスト埋め込み

OpenAIの埋め込みモデルは、最大長を超えるテキストを埋め込むことはできません。最大長はモデルによって異なり、文字列の長さではなく、_tokens_で計測されます。トークン化に慣れていない方は、[How to count to token with tiktoken](How_to_count_tokens_with_tiktoken.ipynb) をご確認ください。

このノートブックでは、モデルの最大文脈長よりも長いテキストを扱う方法を紹介します。ここでは、`text-embedding-ada-002`の埋め込みを使ってデモを行いますが、同じ考え方は他のモデルやタスクにも適用できます。埋め込みについてもっと知りたい方は、[OpenAI Embeddings Guide](https://beta.openai.com/docs/guides/embeddings)をご覧ください。


## 1. モデルコンテキストの長さ

まず、モデルを選択し、APIから埋め込みを取得する関数を定義します。

In [1]:
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt, retry_if_not_exception_type


EMBEDDING_MODEL = 'text-embedding-ada-002'
EMBEDDING_CTX_LENGTH = 8191
EMBEDDING_ENCODING = 'cl100k_base'

# let's make sure to not retry on an invalid request, because that is what we want to demonstrate
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6), retry=retry_if_not_exception_type(openai.InvalidRequestError))
def get_embedding(text_or_tokens, model=EMBEDDING_MODEL):
    return openai.Embedding.create(input=text_or_tokens, model=model)["data"][0]["embedding"]

text-embedding-ada-002`モデルは、`cl100k_base`エンコーディングで8191トークンのコンテキスト長を持っており、その制限を超えるとエラーになることがわかります。

In [2]:
long_text = 'AGI ' * 5000
try:
    get_embedding(long_text)
except openai.InvalidRequestError as e:
    print(e)

This model's maximum context length is 8191 tokens, however you requested 10001 tokens (10001 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.


特にプログラムで多数の埋め込みを処理する場合、このようなエラーを避けたいのは明らかです。しかし、それでも、最大文脈長よりも長いテキストに直面することがあります。以下では、このような長いテキストを扱うための主なアプローチについて説明し、そのレシピを提供します。(1) テキストを最大許容長に切り詰める、(2) テキストをチャンキングし、各チャンクを個別に埋め込む、です。

## 1. 入力テキストを切り捨てる

最も単純な解決策は、入力テキストを許容される最大の長さに切り詰めることです。コンテキストの長さはトークンで測定されるため、切り詰める前にまずテキストをトークン化する必要があります。APIはテキストとトークンの両方の形式で入力を受け付けるので、適切なエンコーディングを使用していることに注意すれば、トークンを文字列形式に変換する必要はありません。以下は、そのような切り捨て関数の例です。

In [3]:
import tiktoken

def truncate_text_tokens(text, encoding_name=EMBEDDING_ENCODING, max_tokens=EMBEDDING_CTX_LENGTH):
    """Truncate a string to have `max_tokens` according to the given encoding."""
    encoding = tiktoken.get_encoding(encoding_name)
    return encoding.encode(text)[:max_tokens]

先ほどの例では、エラーなく動作するようになりました。

In [4]:
truncated = truncate_text_tokens(long_text)
len(get_embedding(truncated))

1536

## 2. 入力テキストをチャンキングする

切り捨ては有効ですが、潜在的に関連性のあるテキストを捨ててしまうという明確な欠点があります。別のアプローチとしては、入力テキストをチャンクに分割し、各チャンクを個別に埋め込む方法があります。その後、チャンクの埋め込みを個別に使用するか、平均化（各チャンクの大きさで重み付け）など何らかの方法で結合することができます。

[Python独自の料理本](https://docs.python.org/3/library/itertools.html#itertools-recipes)から、配列をチャンクに分割する関数を取り上げます。

In [5]:
from itertools import islice

def batched(iterable, n):
    """Batch data into tuples of length n. The last batch may be shorter."""
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while (batch := tuple(islice(it, n))):
        yield batch

ここで、文字列をトークンにエンコードし、それをチャンクに分割する関数を定義します。

In [6]:
def chunked_tokens(text, encoding_name, chunk_length):
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    chunks_iterator = batched(tokens, chunk_length)
    yield from chunks_iterator

最後に、入力テキストが最大文脈長よりも長い場合でも、入力トークンをチャンキングして各チャンクを個別に埋め込むことで、埋め込み要求を安全に処理する関数を書くことができます。平均値`フラグを `True` に設定すると、チャンク埋め込み値の加重平均を返し、`False` に設定すると、チャンク埋め込み値の未修正リストを返すことができます。

In [7]:
import numpy as np


def len_safe_get_embedding(text, model=EMBEDDING_MODEL, max_tokens=EMBEDDING_CTX_LENGTH, encoding_name=EMBEDDING_ENCODING, average=True):
    chunk_embeddings = []
    chunk_lens = []
    for chunk in chunked_tokens(text, encoding_name=encoding_name, chunk_length=max_tokens):
        chunk_embeddings.append(get_embedding(chunk, model=model))
        chunk_lens.append(len(chunk))

    if average:
        chunk_embeddings = np.average(chunk_embeddings, axis=0, weights=chunk_lens)
        chunk_embeddings = chunk_embeddings / np.linalg.norm(chunk_embeddings)  # normalizes length to 1
        chunk_embeddings = chunk_embeddings.tolist()
    return chunk_embeddings

今回も、長い入力テキストに対応できるようになりました。

In [8]:
average_embedding_vector = len_safe_get_embedding(long_text, average=True)
chunks_embedding_vectors = len_safe_get_embedding(long_text, average=False)

print(f"Setting average=True gives us a single {len(average_embedding_vector)}-dimensional embedding vector for our long text.")
print(f"Setting average=False gives us {len(chunks_embedding_vectors)} embedding vectors, one for each of the chunks.")


Setting average=True gives us a single 1536-dimensional embedding vector for our long text.
Setting average=False gives us 2 embedding vectors, one for each of the chunks.


場合によっては、段落の境界や文の境界でチャンクを分割することで、テキストの意味を保つことができるかもしれません。